# API testing met Python

In [ ]:
%%capture
!python -m pip install --upgrade pip pytest pytest-tags ipytest requests 

import pytest
import requests

# alleen nodig in dit notebook
import ipytest
ipytest.autoconfig()
ipytest.config()
pytest_main = lambda opts: (ipytest.run(addopts=opts), ipytest.clean())[0]

In [ ]:
%%html
<style> /* Alle markdown tabellen links uitlijnen */ table { float: left } </style>

# Wat is een API

_Een application programming interface, afgekort tot API, is een verzameling definities op basis waarvan een computerprogramma kan communiceren met een ander programma of onderdeel..._ <sub>bron: <a href=https://nl.wikipedia.org/wiki/Application_programming_interface>Wikipedia</a></sub>

In dit chapter bespreken wij hoe we de API's voor websites kunnen testen met Python.  
We houden het beperkt tot één enkele vorm van: [REST][1]  

[1]: https://nl.wikipedia.org/wiki/Representational_state_transfer

# Hoe gebruik ik een API

Om een API te gebruiken moeten we weten waar we [HTTP messages][1] heen moeten sturen.  
Het adres wat aangesproken moet worden heet een _[API endpoint][2]_.  
Als we weten waar we heen moeten [rest][3] ons de volgende vraag:  
&nbsp; Hoe stuur ik mijn vraag voor data naar de endpoint?

# Hoe gebruik ik een endpoint

Er zijn [vele][4] [manieren][5] om een API te bouwen, daarvoor zijn er ook vele manieren om een API te gebruiken.  
In profressionele organisaties is er documentatie van de API of is het gespecificeerd in een [gestandaardiseerd format][6]
Maar voornamenlijk wordt een API endpoint gebouwt om de volgende data (of combinatie daarvan) te ontvangen.  

1. HTTP Method
1. Headers
1. Body
1. URL + query
1. Cookies


[1]: https://developer.mozilla.org/en-US/docs/Web/HTTP/Messages
[2]: https://smartbear.com/learn/performance-monitoring/api-endpoints/
[3]: https://developer.mozilla.org/en-US/docs/Glossary/REST
[4]: https://octo-woapi.github.io/cookbook/
[5]: https://restcookbook.com/
[6]: https://www.openapis.org/
[7]: https://openapi-generator.tech/

Een overzicht van de meest gebruikte [HTTP Methods][1].

| HTTP Method | Actie op de Server             | Database actie (CRUD) |
|:------------|:-------------------------------|:----------------------|
| POST        | Verstuur data naar de Server   | Create                |
| GET         | Verkrijg de data van de Server | Read                  |
| PUT         | Update de data op de Server    | Update                |
| DELETE      | Verwijder de data op de Server | Delete                |

[1]: https://developer.mozilla.org/en-US/docs/Web/HTTP/Methods

Een overzicht hoe een [URL][1] is opgebouwd.

Voorbeeld: https://www.example.com:443/some/path?single&this=query&key=value#fragment-on-site


| Onderdelen van een URL | Beschrijving                                                                                                | Onderdeel                   |
|:-----------------------|:------------------------------------------------------------------------------------------------------------|:----------------------------|
| Scheme                 | Geeft het protocol aan dat door de URL wordt gebruikt, zoals HTTP, HTTPS, FTP of andere                     | https                       |
| Netloc                 | Bevat de domeinnaam of het IP-adres van de server en kan ook het poortnummer bevatten                       | www.example.com             |
| Port                   | Bevat een nummer dat specificeert welke poort op de server wordt aangesproken                               | 443                         |
| Path                   | Identificeert de specifieke bron die wordt opgevraagd, zoals een webpagina, afbeelding of bestand           | /some/path                  |
| Query                  | Bevat aanvullende parameters die het verzoek wijzigen, zoals zoektermen of filters                          | single&this=query&key=value |
| Fragment               | Identificeert een specifieke sectie of locatie op de opgevraagde bron, zoals een anker tag op een webpagina | fragment-on-site            |


[1]: https://www.rfc-editor.org/rfc/rfc1738

Het is de taak van de client om de request goed op te bouwen om het verwachte response terug te krijgen.  

Na het versturen van een request naar de endpoint geeft de server een response terug.  
De server kan op veel verschillende manieren data terug naar de client sturen.  
De response van een server bevat meestal de volgende data (of combinatie daarvan) 

1. HTTP Status code
1. HTTP Method
1. Headers
1. Body
1. URL + query
1. Cookies

Een overzicht van de [HTTP response status codes][1] die de server terug kan geven

| HTTP response status code | Betekenis         |
|:--------------------------|:------------------|
| 100 - 199                 | Informatief       |
| 200 - 299                 | Request succesvol |
| 300 - 399                 | Request omgeleidt |
| 400 - 499                 | Client error      |
| 500 - 599                 | Server error      |


[1]: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status

# Hoe verstuur ik een request naar een endpoint

In dit chapter houden we het bespreken we één enkele method: [GET][1].  
Browsers doen voornamelijk GET calls naar een server om bijvoorbeeld de [HTML][2] op te vragen van een website. 

De combinatie van een GET request, een URL en een specifieke query kan gebruikt worden om de meest simpele API's aan te roepen.  
Bijvoorbeeld:  https://wttr.in/Veenendaal?m&2&F&lang=nl


# Hoe gebruik ik Python om een request te maken naar een API
Een gebruiksvriendelijke [module][3] om internet verkeer tot stand te brengen komt niet met Python mee geinstalleerd.  
Er zijn externe modules die dit beter kunnen doen, deze zijn gewoon met [PIP][4] te installeren.  

Maar een request maken zonder dependencies is wel mogenlijk, maar het is langdradig ofwel _verbose_.  
Hieronder een voorbeeld.


[1]: https://developer.mozilla.org/en-US/docs/Web/HTTP/Methods/GET
[2]: https://developer.mozilla.org/en-US/docs/Web/HTML
[3]: https://docs.python.org/3/library/urllib.html
[4]: https://pypi.org/project/pip/

In [ ]:
from http.client import HTTPResponse
from urllib.parse import urlencode, urlunparse, ParseResult, parse_qsl
from urllib.request import Request, urlopen


# Uitendelijke url:
#   https://wttr.in/Veenendaal?m2&F&A&lang=nl

# query codes zijn hier uitgeschreven:  https://wttr.in/:help 
query: dict = {
    'm': '',        # metric (SI)
    '2': '',        # current weather + today's + tomorrow's forecast
    'F': '',        # do not show the "Follow" line
    'A': '',        # ignore User-Agent and force ANSI output format (terminal)
    'lang': 'nl'    # Localization
}

# creëer de url
url: str = urlunparse(ParseResult(
    scheme   = 'https',
    netloc   = 'wttr.in',
    path     = '/Veenendaal',
    query    = urlencode(query),    # parse de dict naar een query url part
    params   = None,
    fragment = None,
))

# maak een request aan
request = Request(url, method='GET')

# verstuur het request
response: HTTPResponse = urlopen(request)

# check of de request door de server succesvol is afgehandeld
assert response.code == 200

# lees de data uit de response
data = bytes.decode(response.read())

# weergeef de data
print(data)

## Betere modules om requests mee te maken

[requests][1] is de meest gebruikte module om met Python het internet op te gaan.  
Het is een module die niet [asynchroon][2] kan worden gebruikt.  
[httpx][3] is een andere module voor om requests te maken die wel asynchroon gebruikt kan worden.  

In dit chapter houden we het gemakkelijk en synchroon door [requests][2] te gebruiken.


[1]: https://requests.readthedocs.io/en/latest/
[2]: https://docs.python.org/3/library/asyncio.html
[3]: https://www.python-httpx.org/

In [ ]:
import requests
from requests import Response

# verstuur het request
res: Response = requests.get('https://wttr.in/Veenendaal', params={'m': '', '2': '', 'F': '', 'A': '', 'lang': 'nl'})

# check of de request door de server succesvol is afgehandeld
assert res.ok

# weergeef de data
print(res.text)

# Testen maken in Python

Python heeft zelf een aantal modules ([unittest][1] [doctest][2]) waarmee je testen kan maken.  

De unittest module is oud, ouder dan [PEP-8][3], en gebaseerd op de unittest library van [Java][4]: [JUnit][5].  
Om een simpele testcase te maken moet je gebruik maken van [inheritance][6].  

Doctest wordt gebruikt om programeer snippets in [docstrings][7] te testen.  
De doctest _gimmick_ is niet handig te gebruiken bij grotere testcases.

De meest gebruikte Third-party library om met Python te testen is [pytest][8].  
Deze module is ook gewoon met [PIP][9] te installeren.  
De testcases in pytest is een functie waarvan de naam begint met **test**.  
En met pytest kan je waardes gewoon met [assert][10] testen.  

Voorbeeld van een unittest met pytest.  


[1]:  https://docs.python.org/3/library/unittest.html
[2]:  https://docs.python.org/3/library/doctest.html
[3]:  https://peps.python.org/pep-0008/ "Style Guide for Python Code"
[4]:  https://nl.wikipedia.org/wiki/Java_(programmeertaal)
[5]:  https://junit.org/
[6]:  https://nl.wikipedia.org/wiki/Overerving_(informatica)
[7]:  https://docs.python.org/3/glossary.html#term-docstring
[8]:  https://docs.pytest.org/
[9]:  https://pypi.org/project/pytest/
[10]: https://docs.python.org/3/reference/simple_stmts.html#the-assert-statement

In [ ]:
import pytest

def keer_twee(nummer: int) -> int:
    return int(nummer) * 2

def test_dupliceer_2_en_1_is_6():
    given = keer_twee(2)  # 2 * 2 = 4
    expected = 4
    assert given == expected, f"Error {given = } {expected = }"
    
pytest_main(['--verbosity=1'])

# Combineer de kennis: Unittest + API request

Nu API's en unittests besproken zijn kunnen deze onderwerpen gecombineerd worden voor API tests.  

### De casus
Stel je opdracht is om de _Free tier_ API van [isevenapi.xyz][1] te testen.  
Een andere tester heeft al testcases gemaakt.  
Een test automation engineer heeft al een deel van de testcases geautomatiseerd met Python.  
De manager heeft aangegeven dat het niet uitmaakt hoe je het test, maar de testcases zijn leidend.  
Je wilt de geautomatiseerde testcases gaan uitbreiden.


<!--https://archive.ph/YIXD1-->
[1]: https://isevenapi.xyz/  

## Testcases - Free tier isEven API

| **Title**                                  | isEven is True bij een even nummer                            |
|:-------------------------------------------|:--------------------------------------------------------------|
| Test ID                                    | api_free_001                                                  |
| Requirements                               | Web documentatie                                              |
| Precondities                               | API endpoint moet online zijn                                 |
| Test data                                  | nummer: 2<br/>endpoint: api.isevenapi.xyz/iseven/{nummer}/    |
| **Test steps**                             | **Step result**                                               |
| GET call de API endpoint met de test data  | Response status code is 200 OK<br/> Response body is een JSON |
| Bekijk de JSON                             | JSON heeft een iseven _name_ en _value_                       |
| Check de JSON name: iseven                 | iseven is boolean True                                        |  
  

  
| **Title**                                  | isEven is False bij een oneven nummer                         |
|:-------------------------------------------|:--------------------------------------------------------------|
| Test ID                                    | api_free_002                                                  |
| Requirements                               | Web documentatie                                              |
| Precondities                               | API endpoint moet online zijn                                 |
| Test data                                  | nummer: 1<br/>endpoint: api.isevenapi.xyz/iseven/{nummer}/    |
| **Test steps**                             | **Step result**                                               |
| GET call de API endpoint met de test data  | Response status code is 200 OK<br/> Response body is een JSON |
| Bekijk de JSON                             | JSON heeft een iseven _name_ en _value_                       |
| Check de JSON name: iseven                 | iseven is boolean False                                       |
  

  
| **Title**                                  | isEven is geeft een error bij een negatief nummer                                        |
|:-------------------------------------------|:-----------------------------------------------------------------------------------------|
| Test ID                                    | api_free_003                                                                             |
| Requirements                               | Web documentatie                                                                         |
| Precondities                               | API endpoint moet online zijn                                                            |
| Test data                                  | nummer: -2<br/>endpoint: api.isevenapi.xyz/iseven/{nummer}/                              |
| **Test steps**                             | **Step result**                                                                          |
| GET call de API endpoint met de test data  | Response status code is 401 Unauthorized<br/> Response body is een JSON                  |
| Bekijk de JSON                             | JSON heeft een error _name_ en _value_                                                   |
| Check de JSON name: error                  | error is een string: 'Number out of range. Upgrade to isEven API Premium or Enterprise.' |
  

  
| **Title**                                  | isEven is geeft een error bij een te groot nummer                                      |
|:-------------------------------------------|:---------------------------------------------------------------------------------------|
| Test ID                                    | api_free_004                                                                           |
| Requirements                               | Web documentatie                                                                       |
| Precondities                               | API endpoint moet online zijn                                                          |
| Test data                                  | nummer: 1_000_000<br/>endpoint: api.isevenapi.xyz/iseven/{nummer}/                     |
| **Test steps**                             | **Step result**                                                                        |
| GET call de API endpoint met de test data  | Response status code is 401 Unauthorized<br/> Response body is een JSON                |
| Bekijk de JSON                             | JSON heeft een error _name_ en _value_                                                 |
| Check de JSON name: error                  | error is een string: Number out of range. Upgrade to isEven API Premium or Enterprise. |

## De geautomatiseerde tests

In [ ]:
import pytest

@pytest.mark.tags('api_free_001')
def test_iseven_is_true_bij_een_even_nummer():
    # given
    number = 2
    # when
    response = requests.get(f"https://api.isevenapi.xyz/api/iseven/{number}/")
    data: dict = response.json()
    # then
    assert response.ok, f"{response.status_code} {response.reason} body: {data}" 
    # and
    iseven: bool = data.get('iseven')
    assert iseven and isinstance(iseven, bool), f"iseven should be True, received {iseven}"


@pytest.mark.tags('api_free_002')
def test_iseven_is_false_bij_een_oneven_nummer():
    # given
    number = 1
    # when
    response = requests.get(f"https://api.isevenapi.xyz/api/iseven/{number}/")
    data: dict = response.json()
    # then
    assert response.ok, f"{response.status_code} {response.reason} body: {data}" 
    # and
    iseven: bool = data.get('iseven')
    assert not iseven and isinstance(iseven, bool), f"iseven should be False, received {iseven}"


@pytest.mark.tags('api_free_003')
def test_iseven_is_geeft_een_error_bij_een_negatief_nummer():
    # given
    number = -2
    expected_message = "Number out of range. Upgrade to isEven API Premium or Enterprise."
    # when
    response = requests.get(f"https://api.isevenapi.xyz/api/iseven/{number}/")
    data: dict = response.json()
    # then
    assert not response.ok, f"{response.status_code} {response.reason} body: {data}" 
    # and
    error_message: str = data.get('error')
    assert error_message == expected_message, f"error message was not the same, received: {error_message}"
    # and
    iseven: None = data.get('iseven')
    assert iseven is None, f"iseven should not be available in error response, received: {iseven}"


@pytest.mark.tags('api_free_004')
def test_iseven_is_geeft_een_error_bij_een_te_groot_nummer():
    # given
    number = 1_000_000
    expected_message = ...
    # when
    
    # then
    
    # and
    pytest.fail('Todo!')
    
    
# start de testcase
pytest_main(['--verbosity=1'])

### Oefeningen API testen met Python

* Test de API van [deckofcardsapi.com][1].  
* Maak API tests op basis van de [testcases](./api_testcases/deckofcardsapi_test_cases.md).

[1]: https://deckofcardsapi.com/